In [ ]:
import mysql.connector
import sys

try:
    conn = mysql.connector.connect(
        user='root',
        password='',
        host='localhost',
        database='car_price',
        port=3306
    )

except mysql.connector.Error as e:
    print('Error %s' % e)
    
else:
    print('Berhasil Terkoneksi dengan MySQL server')
    conn.close()

In [ ]:
import mysql.connector
import sys

try:
    conn = mysql.connector.connect(
        user='root',
        password='',
        host='localhost', 
        database='car_price',
        port=3306
    )

    sql = "SELECT `Car Make`, `Car Model`, `Year`, `Engine Size (L)`, `Horsepower`, `Torque (lb-ft)`, `0-60 MPH Time (seconds)`, `Price (in USD)` FROM sport_car_price"
    
    try:
        cur = conn.cursor()
        cur.execute(sql)
            
        results = cur.fetchall()
        
        print("{:<15}{:<25}{:<10}{:<15}{:<15}{:<15}{:<15}{:<10}".format('Car Make','Car Model','Year','Engine Size','Horsepower','Torque','0-60 MPH','Price'))
        
        for row in results:
            car_make = row[0]
            car_model = row[1] 
            year = row[2]
            engine_size = row[3]
            horsepower = row[4]
            torque = row[5]
            zero_to_60 = row[6] 
            price = row[7]

            print("{:<15}{:<25}{:<10}{:<15}{:<15}{:<15}{:<15}{:<10}".format(car_make,car_model,year,engine_size,horsepower,torque,zero_to_60,price))

                            
    except mysql.connector.DataError:
        print("Pengambilan data error")
        
    else:
        cur.close()
        
except mysql.connector.Error as e:
    print('Gagal koneksi ke MySQL: %s' % e)

else:
    print('Berhasil koneksi ke MySQL')
    conn.close()

EKSPLORASI DATA

Memeriksa jumlah baris dan kolom
Menampilkan data teratas
Menampilkan tipe data tiap kolom
Memeriksa missing value / data kosong pada setiap kolom 

In [ ]:
import mysql.connector
import pandas as pd

try:
    conn = mysql.connector.connect(
    user='root',
    password='', 
    host='localhost',
    database='car_price' 
  )
  
    sql = "SELECT * FROM sport_car_price"

    cur = conn.cursor()
    cur.execute(sql)

    results = cur.fetchall()

    df = pd.DataFrame(results, columns=[col[0] for col in cur.description])
    
    baris, kolom = df.shape
    print(f"Data frame memiliki {baris} baris dan {kolom} kolom\n")
    
    jumlah_data_teratas = df.head().to_string(index=False) 
    print(f"Jumlah data teratas:\n{jumlah_data_teratas}\n")
    
    tipe_data = df.dtypes
    print(f"Tipe data setiap kolom:\n{tipe_data}\n")
    
    data_kosong = df.isnull().sum().sum()
    print(f"Jumlah Data kosong pada setiap kolom:{data_kosong}\n")
    
    
except mysql.connector.Error as e:
  print("Error %s" % e)
  
finally:
  conn.close()
  print('Berhasil terkoneksi dengan MySQL server')

DATA CLEANING

Cek Outlier dan Visualisasi Distribusi pada setiap kolom

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

try:
    conn = mysql.connector.connect(
    user='root',
    password='', 
    host='localhost',
    database='car_price' 
  )
  
    sql = "SELECT * FROM sport_car_price"

    cur = conn.cursor()
    cur.execute(sql)

    results = cur.fetchall()

    df = pd.DataFrame(results, columns=[col[0] for col in cur.description])
    
    z = np.abs(stats.zscore(df._get_numeric_data()))
    df = df[(z < 3).all(axis=1)]
    print(df.to_string(index=False))
   
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))

    for i, col in enumerate(df.columns):
        sns.histplot(df[col], ax=axes[i//3, i%3])
        axes[i//3, i%3].set_title(f'Distribusi dari {col}')
    
    plt.tight_layout()
    plt.show()
      
    
except mysql.connector.Error as e:
  print("Error %s" % e)
  
finally:
  conn.close()
  print('Berhasil terkoneksi dengan MySQL server')

ANALISIS DATA